###Instalación Spark, creación de sesión y unidad personal de Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls

drive  sample_data


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget https://downloads.apache.org/spark/spark-3.5.3/spark-3.5.3-bin-hadoop3.tgz

In [ ]:
!wget -q http://apache.mirrors.pair.com/spark/spark-3.5.3/spark-3.5.3-bin-hadoop3.tgz

In [ ]:
!ls

drive  sample_data  spark-3.5.3-bin-hadoop3.tgz


In [ ]:
!tar xf spark-3.5.3-bin-hadoop3.tgz

In [ ]:
!pip install -q findspark
!pip install matplotlib

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.3-bin-hadoop3"
import findspark
findspark.init()
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
findspark.find()

'/content/spark-3.5.3-bin-hadoop3'

In [ ]:
from scipy import stats
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.builder\
        .master("local")\
        .appName("Pyspark_SQL")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [ ]:
spark

DESARROLLO, ejercicio 3:


Cargar el conjunto de datos en un dataframe

In [ ]:
data_path = '/content/drive/MyDrive/TokioSchool/data_curso/'

df = spark.read.options(header=True, inferSchema=True).csv(data_path + 'Air_Traffic_Passenger_Statistics.csv')

¿Cuántas compañías diferentes aparecen en el fichero?

In [ ]:
df.select('Operating Airline').distinct().count()

77

¿Cuántos pasajeros tienen de media los vuelos de cada compañía?

In [ ]:
df_compañia = df.groupBy('Operating Airline').agg (F.mean('Passenger Count').alias('mean_passenger_count'))
df_compañia.show()

+--------------------+--------------------+
|   Operating Airline|mean_passenger_count|
+--------------------+--------------------+
|          Icelandair|              2799.7|
|         Ameriflight|                 5.0|
|      Cathay Pacific|  17121.325581395347|
|          Aeromexico|   5463.822222222222|
|      Etihad Airways|   6476.088235294118|
| Philippine Airlines|  10248.635658914729|
|United Airlines -...|   48915.46750232126|
|    Turkish Airlines|   8162.416666666667|
| Swiss International|   6061.640287769784|
|    Independence Air|              6391.3|
|Miami Air Interna...|             107.375|
|          Air France|  11589.077519379845|
|      Japan Airlines|   6470.332046332046|
|    Midwest Airlines|              3883.0|
|      Atlas Air, Inc|                34.0|
|    JetBlue Airways |   35261.13963963964|
|       China Eastern|   5498.402777777777|
|   Mexicana Airlines|   7993.806451612903|
|         Air Canada |  18251.560109289618|
|       Allegiant Air|          

In [ ]:
top5_compañias = df_compañia.orderBy (F.desc('mean_passenger_count')).limit(5)
top5_compañias.show()

low5_compañias = df_compañia.orderBy (F.asc('mean_passenger_count')).limit(5)
low5_compañias.show()

+------------------+--------------------+
| Operating Airline|mean_passenger_count|
+------------------+--------------------+
| American Airlines|  127164.38970588235|
|Southwest Airlines|   81188.15857605178|
|    Virgin America|   74405.35359116022|
|   United Airlines|   72732.05829596413|
|   Delta Air Lines|   68498.49740932643|
+------------------+--------------------+

+--------------------+--------------------+
|   Operating Airline|mean_passenger_count|
+--------------------+--------------------+
|Evergreen Interna...|                 2.0|
|         Ameriflight|                 5.0|
|      Boeing Company|                18.0|
|      Atlas Air, Inc|                34.0|
|        Xtra Airways|                73.0|
+--------------------+--------------------+



In [ ]:
# Visualizamos con collect para poder mostrar la media de las 77 compañias, ya que con .show() es más visual pero solo muestra 20 filas.
resultados = df_compañia.collect()

# Mostrar los resultados
for fila in resultados:
    print(fila)

Row(Operating Airline='Icelandair', mean_passenger_count=2799.7)
Row(Operating Airline='Ameriflight', mean_passenger_count=5.0)
Row(Operating Airline='Cathay Pacific', mean_passenger_count=17121.325581395347)
Row(Operating Airline='Aeromexico', mean_passenger_count=5463.822222222222)
Row(Operating Airline='Etihad Airways', mean_passenger_count=6476.088235294118)
Row(Operating Airline='Philippine Airlines', mean_passenger_count=10248.635658914729)
Row(Operating Airline='United Airlines - Pre 07/01/2013', mean_passenger_count=48915.46750232126)
Row(Operating Airline='Turkish Airlines', mean_passenger_count=8162.416666666667)
Row(Operating Airline='Swiss International', mean_passenger_count=6061.640287769784)
Row(Operating Airline='Independence Air', mean_passenger_count=6391.3)
Row(Operating Airline='Miami Air International', mean_passenger_count=107.375)
Row(Operating Airline='Air France', mean_passenger_count=11589.077519379845)
Row(Operating Airline='Japan Airlines', mean_passenger_co

Eliminaremos los registros duplicados por el campo “GEO Región”, manteniendo
únicamente aquel con mayor número de pasajeros.

In [ ]:
max_passengers = df.groupBy("GEO Region").agg(F.max("Passenger Count").alias("max_passenger_count"))
max_passengers.show()

+-------------------+-------------------+
|         GEO Region|max_passenger_count|
+-------------------+-------------------+
|             Europe|              48136|
|    Central America|               8970|
|                 US|             659837|
|      South America|               3685|
|             Mexico|              29206|
|        Middle East|              14769|
|             Canada|              39798|
|Australia / Oceania|              12973|
|               Asia|              86398|
+-------------------+-------------------+



Volcaremos los resultados de los dos puntos anteriores a un CSV.

In [ ]:
df_compañia.coalesce(1).write \
  .mode("overwrite") \
  .format("csv") \
  .option("header", "true") \
  .save("/content/drive/MyDrive/TokioSchool/data/")


In [ ]:
max_passengers.coalesce(1).write \
  .mode("overwrite") \
  .format("csv") \
  .option("header", "true") \
  .save("/content/drive/MyDrive/TokioSchool/data_1/")

spark.stop()